In [1]:
from keras.models import load_model

In [2]:
from music21 import converter, instrument, note, chord, stream

In [3]:
import numpy as np
import pickle

In [6]:
with open("notes",'rb') as f:
    notes = pickle.load(f)

In [4]:
model = load_model('model.hdf5')

In [7]:
pitchnames = sorted(set(notes))

In [13]:
n_vocab = len(pitchnames)

In [8]:
int_to_ele = dict((num,ele) for num,ele in enumerate(pitchnames))

In [9]:
ele_to_int = dict((ele,num) for num,ele in enumerate(pitchnames))

## Predictions 

In [10]:
sequence_length = 100
network_input = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i+sequence_length]
    network_input.append([ele_to_int[ch] for ch in seq_in])

In [11]:
len(network_input)

11258

In [26]:
start = np.random.randint(len(network_input)-1)

#Inital pattern
pattern = network_input[start]
prediction_output = []

#generate 200 elements

for note_index in range(200):
    prediction_input  = np.reshape(pattern, (1,len(pattern), 1))
    prediction_input = prediction_input/float(n_vocab)
    
    prediction = model.predict(prediction_input,verbose = 0)
    idx = np.argmax(prediction)
    
    result = int_to_ele[idx]
    prediction_output.append(result)
    pattern.append(idx)
    pattern = pattern[1:]

In [27]:
prediction_output

['G#4',
 'F3',
 '8',
 '3+8',
 'E-5',
 'G#4',
 'E-5',
 'G#5',
 'F2',
 '8',
 'C3',
 'G#4',
 'G#5',
 'F3',
 '3+8',
 'G#3',
 'G#4',
 'E-5',
 'G#5',
 'G2',
 'D5',
 'D3',
 'D5',
 'G3',
 'D5',
 '11+3',
 'D5',
 'D5',
 'C5',
 'G2',
 'D3',
 'B4',
 '2+7',
 'B5',
 'C5',
 'C6',
 'D5',
 'C5',
 'C6',
 '0+3',
 'C6',
 'C3',
 'C2',
 '0+3',
 'C6',
 '0+5',
 '0',
 'G3',
 '0+3',
 'C5',
 '0',
 'G3',
 '0',
 'G3',
 '0+2',
 'C5',
 'C3',
 'C4',
 'G3',
 'C5',
 'G3',
 'B4',
 'G3',
 'C3',
 'C4',
 'C5',
 '0',
 'G3',
 '0',
 'G3',
 'C5',
 '0+2',
 '7+0',
 'C5',
 'E-5',
 'C6',
 'E-3',
 '8',
 'C5',
 'E-5',
 'C6',
 'C6',
 '8',
 'E-3',
 '5+8',
 'C5',
 '0+3',
 'E-3',
 'C5',
 'E-5',
 'C6',
 '8',
 'E-3',
 'E-3',
 '8',
 'E-3',
 'C5',
 'D5',
 '3+8',
 'C5',
 'B4',
 '3+8',
 'C5',
 '8',
 'E-3',
 'E-3',
 '8',
 'E-3',
 'G#2',
 'G#3',
 'E-3',
 '8',
 'F2',
 'F3',
 'G#4',
 'G#5',
 '0+5',
 'G#4',
 'G#5',
 '0+5',
 '0+5',
 'G#4',
 'E-5',
 'G#5',
 '0+5',
 'G#4',
 'E-5',
 'G#5',
 '0+5',
 '0+5',
 '8',
 'C3',
 '0+5',
 '8+0',
 'G#4',
 '0+5',
 

## Create a MIDI File 

In [28]:
offset = 0
output_notes = []

for pattern in prediction_output:
    #if the pattern is a chord
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+')
        temp_notes = []
        
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note)) #Create note object
            new_note.storedInstrument = instrument.Piano()
            temp_notes.append(new_note)
            
        new_chord = chord.Chord(temp_notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    
    
    
    #if the pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note) 
    
    offset += 0.5 #gap is fixed therefore each note or chord will be played for same time

In [29]:
output_notes

[<music21.note.Note G#>,
 <music21.note.Note F>,
 <music21.chord.Chord G#>,
 <music21.chord.Chord E- G#>,
 <music21.note.Note E->,
 <music21.note.Note G#>,
 <music21.note.Note E->,
 <music21.note.Note G#>,
 <music21.note.Note F>,
 <music21.chord.Chord G#>,
 <music21.note.Note C>,
 <music21.note.Note G#>,
 <music21.note.Note G#>,
 <music21.note.Note F>,
 <music21.chord.Chord E- G#>,
 <music21.note.Note G#>,
 <music21.note.Note G#>,
 <music21.note.Note E->,
 <music21.note.Note G#>,
 <music21.note.Note G>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note G>,
 <music21.note.Note D>,
 <music21.chord.Chord B E->,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note C>,
 <music21.note.Note G>,
 <music21.note.Note D>,
 <music21.note.Note B>,
 <music21.chord.Chord D G>,
 <music21.note.Note B>,
 <music21.note.Note C>,
 <music21.note.Note C>,
 <music21.note.Note D>,
 <music21.note.Note C>,
 <music21.note.Note C>,
 <music21.chord.Chord C E-

In [30]:
#create a stream object from the generated objects
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi','test_output.mid')

'test_output.mid'

In [31]:
midi_stream.show('midi')

## Creating and saving more music files in a single directory

In [34]:
for i in range(5):
    
######Creating output for a random input##############    
    start = np.random.randint(len(network_input)-1)

    #Inital pattern
    pattern = network_input[start]
    prediction_output = []

    #generate 200 elements

    for note_index in range(200):
        prediction_input  = np.reshape(pattern, (1,len(pattern), 1))
        prediction_input = prediction_input/float(n_vocab)

        prediction = model.predict(prediction_input,verbose = 0)
        idx = np.argmax(prediction)

        result = int_to_ele[idx]
        prediction_output.append(result)
        pattern.append(idx)
        pattern = pattern[1:]
        
##########################################################

############Creating MIDI File for the output#############

    offset = 0
    output_notes = []

    for pattern in prediction_output:
        #if the pattern is a chord
        if ('+' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('+')
            temp_notes = []

            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note)) #Create note object
                new_note.storedInstrument = instrument.Piano()
                temp_notes.append(new_note)

            new_chord = chord.Chord(temp_notes)
            new_chord.offset = offset
            output_notes.append(new_chord)



        #if the pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note) 

        offset += 0.5 #gap is fixed therefore each note or chord will be played for same time
        
###############################################################

#########creating a stream object and saving it ###############

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi','./Generated Music files/test_output_{}.mid'.format(i+1))
    